# Postwork Sesión 04

## BBDD Walmart

In [37]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [38]:
df_walmart = pd.read_csv('Walmart_Store_sales.csv')
df_walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05/02/2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12/02/2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19/02/2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26/02/2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05/03/2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [39]:
df_walmart.shape

(6435, 8)

In [40]:
print(df_walmart.columns)

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')


In [41]:
X = df_walmart[['Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment']]
Y = df_walmart['Store']

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(3861, 6)
(2574, 6)
(3861,)
(2574,)


### Modelo con polinomio de grado 1 (lineal)

In [43]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

LinearRegression()

In [44]:
y_train_predict = lin_model.predict(X_train)
MSE = mean_squared_error(Y_train,y_train_predict)
print("Entrenamiento: MSE ="+str(MSE))

y_test_predict = lin_model.predict(X_test)
MSE = (mean_squared_error(Y_test, y_test_predict))
print("Pruebas: MSE ="+str(MSE))

Entrenamiento: MSE =139.7952551163223
Pruebas: MSE =134.3529275106685


In [45]:
df_predicciones = pd.DataFrame({'valor_real':Y_test, 'prediccion':y_test_predict, 'diferencia':Y_test-y_test_predict})
df_predicciones = df_predicciones.reset_index(drop = True)
df_predicciones.head(10)

,valor_real,prediccion,diferencia
0,11,15.980705,-4.980705
1,39,18.632078,20.367922
2,7,26.044539,-19.044539
3,17,26.754222,-9.754222
4,33,32.232952,0.767048
5,15,28.772156,-13.772156
6,14,17.681117,-3.681117
7,39,15.802611,23.197389
8,43,23.850783,19.149217
9,29,30.005567,-1.005567


### Modelo con polinomio de grado 2 (parábola)

In [46]:
poly_model = LinearRegression()
poly = PolynomialFeatures(degree=2)
    
Xpolytrain = poly.fit_transform(X_train)
Xpolytest = poly.fit_transform(X_test)
    
poly_model.fit(Xpolytrain, Y_train)
y_train_predict = poly_model.predict(Xpolytrain)
    
MSE = mean_squared_error(Y_train,y_train_predict)
print("Entrenamiento: MSE ="+str(MSE))
    
y_test_predict = poly_model.predict(Xpolytest)
MSE = (mean_squared_error(Y_test, y_test_predict))
print("Pruebas: MSE ="+str(MSE))
    
df_predicciones = pd.DataFrame({'valor_real':Y_test, 'prediccion':y_test_predict, 'diferencia':Y_test-y_test_predict})
df_predicciones = df_predicciones.reset_index(drop = True)
df_predicciones.head(10)

Entrenamiento: MSE =103.86177853443175
Pruebas: MSE =100.0297601779165


,valor_real,prediccion,diferencia
0,11,10.936453,0.063547
1,39,24.146439,14.853561
2,7,22.477394,-15.477394
3,17,28.599949,-11.599949
4,33,32.537338,0.462662
5,15,28.356808,-13.356808
6,14,27.318116,-13.318116
7,39,16.255853,22.744147
8,43,37.601568,5.398432
9,29,30.901505,-1.901505


### Con serie de tiempo

In [47]:
def transformarSerieADataset(serie, elementosPorMuestra):
    dataset = None
    salidasDataset = None
    for counter in range (len(serie)-elementosPorMuestra-1):        
        muestra = np.array([serie[counter:counter+elementosPorMuestra]])        
        salida = np.array([serie[counter+elementosPorMuestra]])
        if dataset is None:
            dataset = muestra
        else:
            dataset = np.append(dataset,muestra,axis = 0)
        if salidasDataset is None:
            salidasDataset = salida    
        else:        
            salidasDataset = np.append(salidasDataset,salida)
    return dataset, salidasDataset

In [48]:
df_timeseries = pd.read_csv('Walmart_Store_sales.csv')
df_timeseries['Weekly_Sales']
serie = df_timeseries['Weekly_Sales'].to_numpy()
    
X, Y = transformarSerieADataset(serie, elementosPorMuestra = 10)

In [49]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

y_train_predict = lin_model.predict(X_train)
MSE = mean_squared_error(Y_train,y_train_predict)
print("Entrenamiento: MSE ="+str(MSE))

y_test_predict = lin_model.predict(X_test)
MSE = (mean_squared_error(Y_test, y_test_predict))
print("Pruebas: MSE ="+str(MSE))

Entrenamiento: MSE =139.7952551163223
Pruebas: MSE =134.3529275106685


In [50]:
df_predicciones = pd.DataFrame({'valor real':Y_test, 'prediccion':y_test_predict, 'diferencia':Y_test-y_test_predict})
df_predicciones = df_predicciones.reset_index(drop = True)
df_predicciones.head(10)

,valor real,prediccion,diferencia
0,11,15.980705,-4.980705
1,39,18.632078,20.367922
2,7,26.044539,-19.044539
3,17,26.754222,-9.754222
4,33,32.232952,0.767048
5,15,28.772156,-13.772156
6,14,17.681117,-3.681117
7,39,15.802611,23.197389
8,43,23.850783,19.149217
9,29,30.005567,-1.005567
